In [3]:
# Automatically reload all imported modules
%load_ext autoreload
%autoreload 2
# Allow to pan and zoom graphs & charts
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exact Interpolation

## Understanding Asteroids

In October 2017, we received an unexpected visit:

<center><img src="assets/oumuamua.jpg" style="height: 380px;"></center>

* The asteroid 'Oumuamua ("scout") came _from another star_
* But how did we know it?


## It's about trajectories

We could tell by _tracking its trajectory_:

<center><img src="assets/trajectory.png" style="height: 380px;"></center>

* Bodies in our solar system have _elliptical orbits_
* Bodies from outside follow _hyperbolic trajectories_

## Taking Advantage of Observations

**So we can:**

* Observe the $(x_0, x_1)$ coordinates at some points...
* ...And see which kind of curve describes it better

**Say we have the following observations $(0, 0)$ is the sun position:**

In [4]:
import pandas as pd

data = pd.DataFrame(columns=['x0', 'x1'], data=[[0.5, 0.1], [-0.55, -0.5]])
data

,x0,x1
0,0.50,0.1
1,-0.55,-0.5


* We have wrapped our data in a pandas `DataFrame` object (i.e. a table)
* [pandas](https://pandas.pydata.org/) is a state-of-the-art library to manage datasets in Python

## Taking Advantage of Observations

**We can plot our observed position on a Cartesian plane (+ the Sun)**

In [5]:
from matplotlib import pyplot as plt

plt.figure(figsize=(9, 3))
plt.scatter(data['x0'], data['x1'], marker='x')
plt.scatter(0, 0, color='0') # sun position
plt.grid(linestyle=':')
plt.axis('equal');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Which Curve?

**We need to choose between two types of curve**

On a (properly chosen) Cartesian plane:

* An ellipsis is described by an equation in the form:

$$
a x_0^2 + b x_1^2 = 1
$$

* ...But that's also the case for a hyperbola


**The difference is all in the _coefficients_:**

* For an ellipsis, both $a$ and $b$ have the same sign
* For a hyperbola, they have opposite signs

## Determining the Coefficients

**Every observation can be translated to a constraint**

For an observation $(x_{i,0}, x_{i,1})$, we have that:
$$
a x_{i,0}^2 + b x_{i,1}^2 = 1
$$

* The values of $x_{i,0}$ and $x_{i,1}$ are _known_
* The values of $a$ and $b$ need to be determined

**For our observations we have that:**
$$
\begin{align}
a (0.5)^2 + b (0.1)^2 &= 1 \\
a (-0.55)^2 + b (-0.5)^2 &= 1 \\
\vdots
\end{align}
$$

* This is a _system of linear equations_
* The variables are $a$ and $b$, i.e. the coefficients in our curve

## Determining the Coefficients

**We can state the system in matrix form**

In general, a linear system can always be rewritten as:
$$
A x = b
$$

* $A$ is the coefficient matrix
* $b$ a vector with the constant terms

**For our asteroid case, we have:**

$$
\left(\begin{array}{ccccc}
(0.5)^2 & (0.1)^2 \\
(-0.55)^2 & (-0.5)^2 \\
\end{array}\right)
\left(\begin{array}{c}
a \\ b
\end{array}\right) =
\left(\begin{array}{c}
1 \\ 1
\end{array}\right)
$$

* Every term in the first matrix column gets multiplied by $a$
* Every term in the second matrix column gets multiplied by $b$

## Solving the System of Linear Equations

**Once in matrix form, the system can be solved via standard algorithms**

* E.g. Gaussian elimination, Singular Value Decomposition, etc.
* We will use an implementation from the numpy module

In [6]:
import numpy as np

A = data**2 # The table with
b = np.ones((2, 1))

w = np.linalg.solve(A, b)
w

array([[ 4.03530895],
       [-0.88272383]])

* The algorithm returns the value of the $a$ and $b$ variables
* From our observations, we can tell that _the curve is a hyperbola_

# Getting General

## Getting General

**It works for any function in the form:**

$$
f(x; w) = \sum_{j = 1}^n w_j \phi_j(x)
$$

* Each $\phi_i(x)$ is called a basis function (it can be anything)
* What we have is a _linear combination_ of basis functions

**Given a set of $m$ observations in the form $(x_i, y_i)$ we can build:**

$$
A =
\left(\begin{array}{cccc}
\phi_1(x_1) & \phi_2(x_1) & \cdots & \phi_n(x_n) \\
\phi_1(x_2) & \phi_2(x_2) & \cdots & \phi_n(x_n) \\
\vdots & \vdots & \vdots & \vdots \\
\phi_1(x_m) & \phi_2(x_m) & \cdots & \phi_n(x_m) \\
\end{array}\right)
\quad
b = 
\left(\begin{array}{c}
y_1 \\
y_2 \\
\vdots \\
y_m \\
\end{array}\right)
$$

* By solving $Aw = b$ we obtain the coefficients $w$

## Softball

**Let's try to characterize a softball throw trajectory**

<center><img src="assets/softball.jpg" style="height: 380px;"></center>

* We will look at the distance $x$ from the thrower
* ...And the ball height $y$ w.r.t. the ground

## Dataset

**Say we have measured the ball position at three points in time**

* We have $x_1 = 0$, $x_2 = 5$, and $x_3 = 10$
* ...And $y_1 = 1.5$, $y_2 = 1.6$, $y_2 = 1.4$

We can store the data in a pandas `DataFrame`

In [7]:
data = pd.DataFrame()
data['x'] = [0, 5, 10]
data['y'] = [1.5, 1.6, 1.4] 
data

,x,y
0,0,1.5
1,5,1.6
2,10,1.4


* For simplicity we will focus on reconstructing the geometric trajectory
* ...And disregard time and speed information

## Choosing a Function

**From basic physics, we know the trajectory is approximately parabolic**

I.e. it will be described by an equation in the form:
$$
y = a x^2 + b x + c
$$

* The expression on the right hand is a linear combination of basis functions
* The basis functions are $x^2$, $x$, and $1$ (for the offset $c$)

**We can therefore formulate a system of linear equations in matrix form**

$$
\left(\begin{array}{ccc}
x_1^2 & x_1 & 1 \\
x_2^2 & x_2 & 1 \\
x_3^2 & x_3 & 1 \\
\end{array}\right)
\left(\begin{array}{c}
a \\
b \\
c \\
\end{array}\right)
=
\left(\begin{array}{c}
y_1 \\
y_2 \\
y_3 \\
\end{array}\right)
$$

* By solving the system we can determine the coefficients $a, b, c$

## Determining the Coefficients

**We can new determine the coefficients**

1. We build the coefficient matrix:

In [8]:
A = pd.DataFrame()
A['x^2'] = data['x']**2
A['x'] = data['x']
A['x^0'] = data['x']**0
A

,x^2,x,x^0
0,0,0,1
1,25,5,1
2,100,10,1


2. We build the constant term vector:

In [9]:
b = data[['y']]

## Determining the Coefficients

3. We solve the system (and plot the curve):

In [10]:
w = np.linalg.solve(A, b)
w

array([[-0.006],
       [ 0.05 ],
       [ 1.5  ]])

In [12]:
x = np.linspace(0, 10)
plt.figure(figsize=(9,3))
plt.plot(x, w[0] * x**2 + w[1] * x + w[2])
plt.scatter(data['x'], data['y'], marker='x', color='tab:red');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …